In [11]:
import pandas as pd
from heapq import *

In [ ]:
direct_df = pd.read_csv('retas.csv', index_col=0, decimal=',').astype(float)
real_df = pd.read_csv('real.csv', index_col=0, decimal=',').astype(float)
lines_df = pd.read_csv('colors.csv', index_col=0)

In [ ]:
direct_df

In [ ]:
class Mapa:
  def __init__(self, tempo_direto, tempo_real, linhas):
    self.tempo_direto = tempo_direto
    self.tempo_real = tempo_real
    self.linhas = linhas
      
  def direct_time(self, estacao_a, estacao_b):
    return self.tempo_direto.at[estacao_a, estacao_b]
  
  def real_time(self, estacao_a, estacao_b):
    return self.tempo_real.at[estacao_a, estacao_b]
  
  def pegar_linhas(self, estacao_a, estacao_b):
    return self.linhas.at[estacao_a, estacao_b]

  def pegar_borda(self, estacao):
    return self.tempo_real[self.tempo_real[estacao].notna()].index.to_list()

In [ ]:
def heuristic(metro, inicio, final): 
  iterador = 1
  estacoes_checadas = []
  queue = [(0, 0, None, inicio)]
  estacao_anterior = {inicio: (None, None)}
  while len(queue) > 0:
    print(f'Fronteira na iteração {iterador}: {[queue]}')
    
    f, g, linha, estacao = heappop(queue)
    estacoes_checadas.append(estacao)
    
    if estacao == final:
      break
    for borda_estacao in metro.pegar_borda(estacao):
      if not borda_estacao in estacoes_checadas:
        border_line = metro.pegar_linhas(estacao, borda_estacao)
        
        if linha == None or linha == border_line: #checa se está na mesma linha pra saber se precisa fazer baldeação (+4min)
          # print(type(g), type(metro.real_time(estacao, borda_estacao)), g, metro.real_time(estacao, borda_estacao))
          borda_g = g + metro.real_time(estacao, borda_estacao)
        else:
          # print(type(g), type(metro.real_time(estacao, borda_estacao)), g, metro.real_time(estacao, borda_estacao))
          borda_g = g + metro.real_time(estacao, borda_estacao) + 4.0
        
        borda_f = borda_g + metro.direct_time(final, borda_estacao)

        if borda_estacao in estacao_anterior.keys():
          if estacao_anterior[borda_estacao][1] > borda_g: # Checa se o tempo anterior do caminho até essa estação é maior que o de agora, caso sim, atualiza qual é a estação anterior
            estacao_anterior[borda_estacao] = (estacao, borda_g)
        else:
          estacao_anterior[borda_estacao] = (estacao, borda_g) # Caso não tenha sido adicionada na fronteira ainda, apenas indica qual é a estação anterior

        heappush(queue, (borda_f, borda_g, border_line, borda_estacao))
    
    iterador += 1
    
  rota = []
  while estacao != None: # itera o caminho reverso até estação inicial utilizando a informação da estação anterior
    rota.insert(0, estacao)
    estacao = estacao_anterior[estacao][0]
  
  print('-'*30)
  print('Rota:', ' -> '.join(rota))
  print(f'Duração: {g:.2f} minutos')

In [42]:
metro_de_paris = Mapa(direct_df, real_df, lines_df)

In [45]:
# primeiro exemplo
heuristic(metro_de_paris, 'E1', 'E11')

Fronteira na iteração 1: [[(0, 0, None, 'E1')]]
Fronteira na iteração 2: [[(82.0, 20.0, 'blue', 'E2')]]
Fronteira na iteração 3: [[(88.8, 44.0, 'yellow', 'E9'), (115.0, 37.0, 'blue', 'E3'), (101.4, 31.0, 'yellow', 'E10')]]
Fronteira na iteração 4: [[(72.4, 72.4, 'red', 'E11'), (101.4, 31.0, 'yellow', 'E10'), (144.8, 66.8, 'red', 'E3'), (127.60000000000001, 63.2, 'yellow', 'E8'), (115.0, 37.0, 'blue', 'E3')]]
------------------------------
Rota: E1 -> E2 -> E9 -> E11
Duração: 72.40 minutos
